## PROSPECT BASE

To train our model, we will be using prospects who applied to a given credit card on the RBC public site. Note, it doesn't matter if they were approved or not, it's important that they applied. 

##### Timing 
We want to time how long these programs take to run. We are interested both in clock time (real time) and wall time (CPU time). 

In [1]:
import time 

start_time = time.time()
start_cpu_time = time.process_time()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
25603,application_1752182618510_216485,pyspark,idle,Link,Link,359335502,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Set Up

In [2]:
import os
import numpy as np
import calendar

import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql.functions import collect_list, regexp_replace, lower
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.functions import year, month, dayofmonth, to_date, trim, concat, col, lit
from functools import reduce

import datetime 
from datetime import datetime as dt
from dateutil.relativedelta import *
import pandas as pd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Load in Applicant Data

Define the look back period as the 12 months prior to current date. (!) The exception being that GA4 was only implemented in July 2024 (!)

__We also want to apply the following filters to these applicants:__ 
- Personal, not business client; approved product is a personal banking account (PROD_APPRVD = P)
- Not an existing client (CLNT_TENURE_SEG_CD != EXISTING) 

##### Function to Pull Months

In [3]:
def month_end_list(num_months): 

    #Define period start and end based on today's date 
    today = datetime.date.today()
    period_start = today.replace(day=1) + relativedelta(months=-num_months)
    period_end = today.replace(day=1)

    #Make sure we keep only the period that we have GA4 rolled out - before July 31, 2024
    #We will be looking at previous sessions and logins 30 days before app
    #Therefore, we really care about the period after Aug 2024
    #if (period_start < datetime.date(2024,8,1)): #August
    #   period_start = datetime.date(2024,8,1)

    #Create list of month ends
    dtrange = pd.date_range(start=period_start, end=period_end, freq='d')
    months = pd.Series(dtrange.month)
    starts, ends = months.ne(months.shift(1)), months.ne(months.shift(-1))
    df = pd.DataFrame({'month_starting_date': dtrange[starts].strftime('%Y-%m-%d'),
                       'month_ending_date': dtrange[ends].strftime('%Y-%m-%d')})
    
    me_list = list(df['month_ending_date'])[:-1]
    return me_list 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
print(month_end_list(11))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['2024-08-31', '2024-09-30', '2024-10-31', '2024-11-30', '2024-12-31', '2025-01-31', '2025-02-28', '2025-03-31', '2025-04-30', '2025-05-31', '2025-06-30']

##### Function to Check Existing File Paths

In [5]:
def check_folder(path):

    # Access Hadoop FileSystem
    hadoop_fs = spark._jsc.hadoopConfiguration()
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(hadoop_fs)
    
    # List subfolders
    files = fs.listStatus(spark._jvm.org.apache.hadoop.fs.Path(path))
    subfolders = [file.getPath().getName() for file in files if file.isDirectory()]
        
    return subfolders

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Function to Import Applicant Data for Each Month in Range

In [6]:
subfolders = check_folder("/prod/14460/app/UQ20/data/lab/POMA/anaurosevic/cdn0_cards_affinity/appl_fact/")
print(subfolders)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['2024-08-31', '2024-09-30', '2024-10-31', '2024-11-30', '2024-12-31', '2025-01-31', '2025-02-28', '2025-03-31', '2025-04-30', '2025-05-31', '2025-06-30']

In [7]:
months_to_download = list(set(month_end_list(10)) - set(subfolders))
print(months_to_download)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[]

In [8]:
#Import for each month end date 
def import_appl_fact(me_list): 

    #Loop over list of month ends
    for i in me_list: 
        print(i)

        #Set file name
        folder_path =  "/prod/14460/app/UQ20/data/lab/POMA/anaurosevic/cdn0_cards_affinity/appl_fact/"
        file_path = folder_path + str(i)
    
        #Create date filter string 
        date_filter_string = "ME_DT == '" + str(i) + "'"
    
        appl_fact = spark.read.option(
            "basePath","/prod/sz/tsz/00104/cc/D3CTA_TAPPL_FACT_DLY_COMPL/").load(
            "/prod/sz/tsz/00104/cc/D3CTA_TAPPL_FACT_DLY_COMPL/").filter(
            date_filter_string).withColumn(
            "ep_reference_id", F.substring(F.trim(F.col("APPLCNT_NUM")),0,8)).filter(
            #Personal not business client
            "trim(PROD_APPRVD)= 'P'").filter(
            #Not an existing client at the time of application
            "trim(CLNT_TENURE_SEG_CD) != 'EXISTING'").select(
            "CLNT_NO","APPLCNT_NUM","ep_reference_id","PSTCD","APP_RCV_DT",
        ).distinct()
            
        appl_fact.coalesce(1).write.mode("overwrite").parquet(file_path)

    return appl_fact 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
#import_appl_fact(months_to_download)
import_appl_fact(['2025-06-30']) #Just for testing

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2025-06-30
DataFrame[CLNT_NO: int, APPLCNT_NUM: decimal(13,0), ep_reference_id: string, PSTCD: string, APP_RCV_DT: date]

##### Combine Files 

In [10]:
#Import 
base_path = "/prod/14460/app/UQ20/data/lab/POMA/anaurosevic/cdn0_cards_affinity/appl_fact/"

for i in range (0,len(month_end_list(11))): 

    dynamic_path = base_path + str(month_end_list(11)[i])
    month = spark.read.load(dynamic_path)
    
    if i==0:
        appl_12m = month
    else:
        appl_12m = appl_12m.union(month)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
print(appl_12m.count())
print(appl_12m.distinct().count()) #Yay

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

344457
344457

#### Load in E-commerce Data
This tells us who who clicked submit on the credit cards application flow via the public site. We only want to keep customers that applied through the public site (basically exclude situations where they went through an advisor or some other form).

##### Function to Import E-Commerce Data for Each Month in Range

In [12]:
subfolders = check_folder("/prod/14460/app/UQ20/data/lab/POMA/anaurosevic/cdn0_cards_affinity/ecommerce/") #These are the files we have already downloaded :) 
print(subfolders)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['2024-08-31', '2024-09-30', '2024-10-31', '2024-11-30', '2024-12-31', '2025-01-31', '2025-02-28', '2025-03-31', '2025-04-30', '2025-05-31', '2025-06-30']

In [13]:
months_to_download = list(set(month_end_list(10)) - set(subfolders))
print(months_to_download)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[]

In [14]:
def import_ecommerce_data(me_list): 

    date = dt.strptime(me_list[0], '%Y-%m-%d').date()

    #Loop over list of month ends
    for i in range(0,len(me_list)): 
        print(me_list[i])

        #Set file name for saving 
        save_folder_path =  "/prod/14460/app/UQ20/data/lab/POMA/anaurosevic/cdn0_cards_affinity/ecommerce/"
        save_file_path =  save_folder_path + str(me_list[i])

        #Location of files 
        data_folder_path = "/prod/sz/tsz/00198/data/ga4-ecommerce/"
        date = dt.strptime(me_list[i], '%Y-%m-%d').date()
        date_filter_string = "YEAR=" + str(date.strftime('%Y')) + "/Month=" + str(date.strftime('%m')) + "/"
        data_file_path = data_folder_path + date_filter_string

        #Import data 
        ecommerce = spark.read.load(
            data_file_path).filter(
            "ep_lob = 'credit cards' and ep_content_group = 'credit cards : cardapp'").filter(
            #Made it to the final submission step
            "ep_step_name = 'step_finished'").filter(
            #Not missing application ID
            "ep_reference_id is not null").filter(
            #Make sure we don't have internal traffic - e.g., branch computer
            "ep_traffic_type is null").withColumn(
            #Format CC choice
            "product_code", F.substring("it_item_id",3,3)).select(  
            "user_pseudo_id", F.trim(F.col("ep_reference_id")).alias("ep_reference_id"), 
            "event_date", "user_session_id", "product_code").distinct()
                
        ecommerce.coalesce(1).write.mode("overwrite").parquet(save_file_path)

    return ecommerce 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#import_ecommerce_data(months_to_download)
import_ecommerce_data(['2025-06-30']) #Just for testing

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2025-06-30
DataFrame[user_pseudo_id: string, ep_reference_id: string, event_date: date, user_session_id: string, product_code: string]

##### Combine Files 

In [16]:
#Import 
base_path = "/prod/14460/app/UQ20/data/lab/POMA/anaurosevic/cdn0_cards_affinity/ecommerce/"

for i in range (0,len(month_end_list(11))): 

    dynamic_path = base_path + str(month_end_list(11)[i])
    month = spark.read.load(dynamic_path)
    
    if i==0:
        ecommerce_12m = month
    else:
        ecommerce_12m = ecommerce_12m.union(month)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
print(ecommerce_12m.count())
print(ecommerce_12m.distinct().count())
#Great, no duplicates to worry about :) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

184506
184506

#### Join Tables to Identify Prospect Base
Join the two tables so that we can ensure that customers who made it to the CC submission point (via e-commerce) are not existing clients. 

In [18]:
#Join applicant table to e-commerce table via ep_reference_id
#Ignore existing clients 
ga_to_appl = ecommerce_12m.join(appl_12m, [
    (ecommerce_12m.ep_reference_id == appl_12m.ep_reference_id) & (ecommerce_12m.event_date <= appl_12m.APP_RCV_DT)
]).drop(ecommerce_12m.ep_reference_id)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
ga_to_appl.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_pseudo_id: string (nullable = true)
 |-- event_date: date (nullable = true)
 |-- user_session_id: string (nullable = true)
 |-- product_code: string (nullable = true)
 |-- CLNT_NO: integer (nullable = true)
 |-- APPLCNT_NUM: decimal(13,0) (nullable = true)
 |-- ep_reference_id: string (nullable = true)
 |-- PSTCD: string (nullable = true)
 |-- APP_RCV_DT: date (nullable = true)

In [20]:
ga_to_appl.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+----------+--------------------------------+------------+---------+-----------+---------------+------+----------+
|user_pseudo_id       |event_date|user_session_id                 |product_code|CLNT_NO  |APPLCNT_NUM|ep_reference_id|PSTCD |APP_RCV_DT|
+---------------------+----------+--------------------------------+------------+---------+-----------+---------------+------+----------+
|577651241.1725152216 |2024-08-31|577651241.1725152216_1725152216 |IAV         |459301206|173217027  |17321702       |J1L2M8|2024-09-03|
|2113191595.1723157405|2024-08-08|2113191595.1723157405_1723166451|MC4         |405716309|172611469  |17261146       |T5T7C2|2024-08-08|
|895200250.1724852347 |2024-08-28|895200250.1724852347_1724852347 |MV1         |459238028|173112103  |17311210       |H7X2W7|2024-08-28|
|1195917261.1724503351|2024-08-24|1195917261.1724503351_1724503350|CLO         |474560976|173013301  |17301330       |H4V2H8|2024-08-24|
|1875057210.1724503117|2024-08-24|1875057

In [21]:
ga_to_appl.withColumn('APPLCNT_NUM_TRUNC',F.col('ep_reference_id')).select(
    'CLNT_NO','user_pseudo_id','event_date','APP_RCV_DT','APPLCNT_NUM','APPLCNT_NUM_TRUNC','ep_reference_id').filter(
    "ep_reference_id='17261146'").show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------------------+----------+----------+-----------+-----------------+---------------+
|CLNT_NO  |user_pseudo_id       |event_date|APP_RCV_DT|APPLCNT_NUM|APPLCNT_NUM_TRUNC|ep_reference_id|
+---------+---------------------+----------+----------+-----------+-----------------+---------------+
|405716309|2113191595.1723157405|2024-08-08|2024-08-08|172611469  |17261146         |17261146       |
+---------+---------------------+----------+----------+-----------+-----------------+---------------+

In [22]:
ga_to_appl.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

160575

In [23]:
#Grab subset of columns of interest ~
new_clients = ga_to_appl.withColumnRenamed(
    "PSTCD","postal_code").select(
    "user_pseudo_id","user_session_id","ep_reference_id",F.col("CLNT_NO").alias('clnt_no'),"postal_code","product_code","event_date").persist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
new_clients.show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+--------------------------------+---------------+---------+-----------+------------+----------+
|user_pseudo_id       |user_session_id                 |ep_reference_id|clnt_no  |postal_code|product_code|event_date|
+---------------------+--------------------------------+---------------+---------+-----------+------------+----------+
|577651241.1725152216 |577651241.1725152216_1725152216 |17321702       |459301206|J1L2M8     |IAV         |2024-08-31|
|2113191595.1723157405|2113191595.1723157405_1723166451|17261146       |405716309|T5T7C2     |MC4         |2024-08-08|
|895200250.1724852347 |895200250.1724852347_1724852347 |17311210       |459238028|H7X2W7     |MV1         |2024-08-28|
|1195917261.1724503351|1195917261.1724503351_1724503350|17301330       |474560976|H4V2H8     |CLO         |2024-08-24|
|1875057210.1724503117|1875057210.1724503117_1724503116|17301348       |475088951|J8X3T9     |MC4         |2024-08-24|
+---------------------+-------------------------

In [25]:
new_clients.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

160575

#### Fix Inconsistencies
We would expect each row to be unique: one user, one session, one product, one date. We also shouldn't see cases where there are multiple rows per user_pseudo_id. Therefore, we are removing strange cases like the following: 
- __(A) Missing user_pseudo_id or user_session_id:__ We won't be able to link these customers to their session details and it is unclear why this would ever be missing
- __(B) Multiple applications or cards per session (unique at the session level):__ It could be the following reasons: (1) They are scammers that are re-submitting applications for the same card multiple times; (2) They could be households with diff people applying within the same session off the same device; (3) It could be a branch or public computer during which multiple people are submitting. All three cases are problematic. We will only keep prospects who have 1 application for a single CC in the session.
- __(C) Multiple applications or sessions for the same device (unique at the user level):__ Multiple sessions for the same device could be if they get declined or multiple households. We will make the assumption that we should only consider the first application for each device.

##### (A) Missing user_pseudo_id or user_session_id

In [26]:
print(new_clients.filter("user_pseudo_id is null").count())
print(new_clients.filter("user_session_id == '_'").count())
#There are 1,807 people who don't have a user pseudo id nor a session id - let's drop these, not sure why they are happening 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2535
2535

In [27]:
df = new_clients.filter("user_pseudo_id is not null")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
print(df.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

158040

##### (B) Multiple applications or cards per session (unique at the session level)

In [29]:
#Keep only people who had one application within the session 
one_application = df.groupBy(["user_pseudo_id","user_session_id"]).count().filter("count==1").select('user_session_id')
one_application.show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------------+
|user_session_id                 |
+--------------------------------+
|2113191595.1723157405_1723166451|
|1416482687.1723834437_1723834437|
|986296509.1722804916_1724300447 |
|1518093279.1722953315_1722953317|
|593523526.1724862317_1724872518 |
+--------------------------------+
only showing top 5 rows

In [30]:
#Merge 
df_one_app = df.join(one_application, 
                  [df.user_session_id == one_application.user_session_id],"inner").drop(
    one_application.user_session_id).select(
    "user_pseudo_id","user_session_id","clnt_no","event_date","postal_code","product_code").distinct()
df_one_app.count()
#Now it should be unique at the session level - within each session, the customer only applied once! 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

152434

In [31]:
df_one_app.show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+--------------------------------+---------+----------+-----------+------------+
|user_pseudo_id       |user_session_id                 |clnt_no  |event_date|postal_code|product_code|
+---------------------+--------------------------------+---------+----------+-----------+------------+
|778022095.1733164379 |778022095.1733164379_1733188957 |410120158|2024-12-02|T1S0B3     |MC4         |
|987097481.1735335765 |987097481.1735335765_1735335764 |579028911|2024-12-27|T2X0R9     |MC4         |
|1851791527.1733525494|1851791527.1733525494_1733525494|410211783|2024-12-06|T6C1A9     |MC1         |
|609292779.1734068126 |609292779.1734068126_1734068126 |359560794|2024-12-13|N6K3X8     |CLO         |
|1857597691.1735334679|1857597691.1735334679_1735342894|343350781|2024-12-27|L4H1Y9     |MC4         |
+---------------------+--------------------------------+---------+----------+-----------+------------+
only showing top 5 rows

In [32]:
#Grab session timestamp! 
df_one_app = df_one_app.withColumn(
    "session_timestamp", F.from_unixtime(F.split(F.col("user_session_id"), "_").getItem(1)))
df_one_app.show(5,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+--------------------------------+---------+----------+-----------+------------+-------------------+
|user_pseudo_id       |user_session_id                 |clnt_no  |event_date|postal_code|product_code|session_timestamp  |
+---------------------+--------------------------------+---------+----------+-----------+------------+-------------------+
|1728862783.1723587929|1728862783.1723587929_1723591251|459011805|2024-08-13|H8Y1Z4     |IOP         |2024-08-13 19:20:51|
|1214624237.1722969735|1214624237.1722969735_1722969735|405637166|2024-08-06|T3N1B5     |CLO         |2024-08-06 14:42:15|
|965854520.1723852609 |965854520.1723852609_1723852609 |405920026|2024-08-16|T8N6W7     |MC4         |2024-08-16 19:56:49|
|699752771.1723014368 |699752771.1723014368_1723014377 |405655366|2024-08-07|V5E3C4     |ION         |2024-08-07 03:06:17|
|92876622.1724802418  |92876622.1724802418_1724802417  |406195149|2024-08-27|T4L1V1     |MC1         |2024-08-27 19:46:57|
+---------------

##### (C) Multiple applications or sessions for the same device (unique at the user level)

In [33]:
#In these cases, we'll take the first credit card application as the event date and CC for that client
df_one_app_first = df_one_app.withColumn(
    "rank", F.row_number().over(Window.partitionBy("user_pseudo_id").orderBy("session_timestamp"))
).filter("rank = 1").drop("rank")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Add province/territory breakdown 
Province is important especially for certain cards that are based on region: moi - QC, Westjet - BC.

In [34]:
#Let's also grab the province - province level details are important 
#Source: https://www12.statcan.gc.ca/census-recensement/2021/ref/dict/tab/index-eng.cfm?ID=T1_9
df_one_app_first = df_one_app_first.withColumn(
            "province",
            F.when(
                F.substring("postal_code",1,1)=='A','NL').when(
                F.substring("postal_code",1,1)=='B','NS').when(
                F.substring("postal_code",1,1)=='C','PE').when(
                F.substring("postal_code",1,1)=='E','NB').when(
                F.substring("postal_code",1,1).isin('G','H','J'), 'QC').when(
                F.substring("postal_code",1,1).isin('K','L','M','N','P'), 'ON').when(
                F.substring("postal_code",1,1)=='R', 'MB').when(
                F.substring("postal_code",1,1)=='S', 'SK').when(
                F.substring("postal_code",1,1)=='T', 'AB').when(
                F.substring("postal_code",1,1)=='V', 'BC').when(
                F.substring("postal_code",1,1)=='X', 'NTNU').when(
                F.substring("postal_code",1,1)=='Y', 'YT').otherwise(
                "unk"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Remove Devices with Client Login in Last 30 Days

In [35]:
new_cards = df_one_app_first.withColumn(
    "rank", F.row_number().over(Window.partitionBy("user_pseudo_id").orderBy("event_date"))
).filter("rank = 1").drop("rank").withColumn(
    "date_lag", F.date_add(F.col("event_date"), -1) # need a one day delay due to above reason
).withColumnRenamed(
    "event_date", "card_sale_date")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

##### Function to Import Visitor Data for Each Month in Range
Important: Make sure we have one additional month prior!

In [36]:
print(month_end_list(12))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['2024-07-31', '2024-08-31', '2024-09-30', '2024-10-31', '2024-11-30', '2024-12-31', '2025-01-31', '2025-02-28', '2025-03-31', '2025-04-30', '2025-05-31', '2025-06-30']

In [37]:
subfolders = check_folder("/prod/14460/app/UQ20/data/lab/POMA/anaurosevic/cdn0_cards_affinity/visitor/")
print(subfolders)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['2024-07-31', '2024-08-31', '2024-09-30', '2024-10-31', '2024-11-30', '2024-12-31', '2025-01-31', '2025-02-28', '2025-03-31', '2025-04-30', '2025-05-31', '2025-06-30']

In [38]:
months_to_download = list(set(month_end_list(11)) - set(subfolders))
print(months_to_download)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[]

In [39]:
def import_visitors_data(me_list): 
    
    #Loop over list of month ends
    for i in range(0,len(me_list)): 
        print(me_list[i])

        #Set file name for saving 
        save_folder_path =  "/prod/14460/app/UQ20/data/lab/POMA/anaurosevic/cdn0_cards_affinity/visitor/"
        save_file_path =  save_folder_path + str(me_list[i]) 

        #Location of files 
        data_folder_path = "/prod/sz/tsz/00198/data/ga4-visitor/"
        date = dt.strptime(me_list[i], '%Y-%m-%d').date()
        date_filter_string = "YEAR=" + str(date.strftime('%Y')) + "/Month=" + str(date.strftime('%m'))
        data_file_path = data_folder_path + date_filter_string + "/*"

        #Import data 
        visitors = spark.read.option(
            "basePath",data_folder_path).load(
            data_file_path).withColumn(
            "visitor_table_date", 
            F.to_date(F.concat(F.col("YEAR"), F.lit("-"), F.col("Month"), F.lit("-"), F.col("Day")), "yyyy-MM-dd")).select(
            "user_pseudo_id","visitor_table_date").distinct()
                
        visitors.coalesce(1).write.mode("overwrite").parquet(save_file_path)

    return visitors 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
#import_visitors_data(months_to_download)
import_visitors_data(['2025-06-30']) #Just to test 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2025-06-30
DataFrame[user_pseudo_id: string, visitor_table_date: date]

##### Combine Files 

In [41]:
#Import 
base_path = "/prod/14460/app/UQ20/data/lab/POMA/anaurosevic/cdn0_cards_affinity/visitor/"

for i in range (0,len(month_end_list(12))): 

    dynamic_path = base_path + str(month_end_list(12)[i])
    month = spark.read.load(dynamic_path)
    
    if i==0:
        visitors_12m = month
    else:
        visitors_12m = visitors_12m.union(month)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### Join Tables to Exclude Devices with Previous Login

In [42]:
new_cards = new_cards.join(visitors_12m, 
    on=[(new_cards.user_pseudo_id == visitors_12m.user_pseudo_id) & (visitors_12m.visitor_table_date == new_cards.date_lag)], 
    how='left_anti').drop(
    new_cards.date_lag).drop(
    visitors_12m.visitor_table_date)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
new_cards.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

144577

In [44]:
new_cards.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+--------------------------------+---------+--------------+-----------+------------+-------------------+--------+
|user_pseudo_id       |user_session_id                 |clnt_no  |card_sale_date|postal_code|product_code|session_timestamp  |province|
+---------------------+--------------------------------+---------+--------------+-----------+------------+-------------------+--------+
|1001168409.1739291498|1001168409.1739291498_1739291498|173608217|2025-02-11    |K1T1W2     |ION         |2025-02-11 11:31:38|ON      |
|1012026430.1743550318|1012026430.1743550318_1743550318|412655466|2025-04-01    |T2E5Y7     |MC4         |2025-04-01 19:31:58|AB      |
|1013706846.1741089233|1013706846.1741089233_1741089232|461744328|2025-03-04    |E2E4S2     |MC4         |2025-03-04 06:53:52|NB      |
|1021530018.1742945154|1021530018.1742945154_1742945153|462046012|2025-03-25    |J3Y3E1     |CLO         |2025-03-25 19:25:53|QC      |
|1026676575.1732803848|1026676575.1732803848_173

#### CCs of Interest

In [45]:
cb = new_cards

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
cb.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

144577

In [47]:
credit_cards = ["CLO","IAV","ION","MC4","IOP","GCP","MC1","AVP","MC2","MCP","MV1","PLT","GUS","BAP"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
cb.show(3,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+--------------------------------+---------+--------------+-----------+------------+-------------------+--------+
|user_pseudo_id       |user_session_id                 |clnt_no  |card_sale_date|postal_code|product_code|session_timestamp  |province|
+---------------------+--------------------------------+---------+--------------+-----------+------------+-------------------+--------+
|1001168409.1739291498|1001168409.1739291498_1739291498|173608217|2025-02-11    |K1T1W2     |ION         |2025-02-11 11:31:38|ON      |
|1012026430.1743550318|1012026430.1743550318_1743550318|412655466|2025-04-01    |T2E5Y7     |MC4         |2025-04-01 19:31:58|AB      |
|1013706846.1741089233|1013706846.1741089233_1741089232|461744328|2025-03-04    |E2E4S2     |MC4         |2025-03-04 06:53:52|NB      |
+---------------------+--------------------------------+---------+--------------+-----------+------------+-------------------+--------+
only showing top 3 rows

In [49]:
cb_ccs_of_interest = cb.filter(F.col('product_code').isin(credit_cards))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
cb_ccs_of_interest.groupBy('product_code').count().orderBy(F.desc('count')).show(50,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+
|product_code|count|
+------------+-----+
|MC4         |39666|
|CLO         |27230|
|MC1         |20603|
|IAV         |16631|
|ION         |12674|
|IOP         |7027 |
|MC2         |6232 |
|MV1         |4234 |
|GCP         |3247 |
|AVP         |1430 |
|MCP         |1369 |
|PLT         |505  |
|BAP         |284  |
|GUS         |257  |
+------------+-----+

##### Final checks

In [51]:
cb_ccs_of_interest.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

141389

In [52]:
cb_ccs_of_interest.agg(F.min(F.col('card_sale_date')).alias('min_date'), F.max(F.col('card_sale_date')).alias('max_date')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+
|  min_date|  max_date|
+----------+----------+
|2024-08-01|2025-06-27|
+----------+----------+

In [53]:
cb_ccs_of_interest.show(10) #Yay!! :) 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+---------+--------------+-----------+------------+-------------------+--------+
|      user_pseudo_id|     user_session_id|  clnt_no|card_sale_date|postal_code|product_code|  session_timestamp|province|
+--------------------+--------------------+---------+--------------+-----------+------------+-------------------+--------+
|1001168409.173929...|1001168409.173929...|173608217|    2025-02-11|     K1T1W2|         ION|2025-02-11 11:31:38|      ON|
|1012026430.174355...|1012026430.174355...|412655466|    2025-04-01|     T2E5Y7|         MC4|2025-04-01 19:31:58|      AB|
|1013706846.174108...|1013706846.174108...|461744328|    2025-03-04|     E2E4S2|         MC4|2025-03-04 06:53:52|      NB|
|1021530018.174294...|1021530018.174294...|462046012|    2025-03-25|     J3Y3E1|         CLO|2025-03-25 19:25:53|      QC|
|1026676575.173280...|1026676575.173280...|359271624|    2024-11-28|     L2G5S9|         MC2|2024-11-28 09:24:07|      ON|
|1031188694.1743

#### Save File 

In [54]:
cb_ccs_of_interest.coalesce(1).write.mode("overwrite").parquet("/prod/14460/app/UQ20/data/lab/POMA/anaurosevic/cdn0_cards_affinity/prospect_base/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--- END PROGRAM ---

In [55]:
#Timing summary
end_time = time.time()
end_cpu_time = time.process_time()

real_time_elapsed = end_time - start_time
cpu_time_elapsed = end_cpu_time - start_cpu_time

print(f"Real time: {real_time_elapsed:.2f} seconds")
print(f"CPU time: {cpu_time_elapsed:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Real time: 7970.82 seconds
CPU time: 1.70 seconds